In [2]:
import os
import sys
import pandas as pd
import tqdm

sys.path.append("..")
#os.chdir(os.path.join(os.getcwd(), ".."))

from src.rewe_data_scraper import load_driver, go_to_next_category, scrape_product_category_data, go_next_page, go_back
from src.rewe_data_scraper import get_number_of_product_categories, get_number_of_pages, random_sleep

output_path = "data/rewe_dataset.csv"
driver = load_driver(config_path="config/rewe_browser_config.json")

visited_category_names = []
page_dfs = []
n_categories = get_number_of_product_categories(driver)
i_categories = 0

category_bar = tqdm.tqdm(total=n_categories, desc="Product Categories", leave=True)

while True:
    random_sleep()
    next_category_name = go_to_next_category(driver, visited_category_names)
    if next_category_name == False:
        break
    visited_category_names.append(next_category_name)

    i_categories += 1
    category_bar.update(1)
    #print(f"\nProcessing Category: {i_categories}/{n_categories}\n")

    i_pages = 0
    n_pages = get_number_of_pages(driver)

    page_bar = tqdm.tqdm(total=n_pages, desc="Pages", leave=False)

    while True:
        random_sleep()
        i_pages += 1
        page_bar.update(1)
        #print(f"\nProcessing Page: {i_pages}/{n_pages}\n")

        page_df = scrape_product_category_data(driver.page_source, next_category_name)
        page_dfs.append(page_df)

        random_sleep()
        if go_next_page(driver) == False:
            break

    while True:
        if go_back(driver) == False:
            break

dfs = pd.concat(page_dfs, ignore_index=True)
dfs.to_csv(output_path)
dfs.reset_index(drop=True, inplace=True)
driver.close()

Product Categories:   5%|▍         | 1/21 [00:03<01:01,  3.06s/it]

Product Categories:  10%|▉         | 2/21 [00:17<03:05,  9.75s/it]








KeyboardInterrupt: 

In [ ]:
# with tqdm(total=len(CATEGORIES), desc='Categories Progress', position=0) as categories_pbar:
#         total_pages = int([button for button in buttons if '>' in button.text][-2].text)
#         with tqdm(total=total_pages, desc=f'Page Progress for {category}', position=1, leave=False) as page_pbar:
